## Knapsack

In [60]:
import numpy as np
import json
import re


In [102]:
test_case_1 = """
knapsack([{ name:'map', weight:9, value:150 }, { name:'compass', weight:13, value:35 }, { name:'water', weight:153, value:200 }, { name:'sandwich', weight:50, value:160 }, { name:'glucose', weight:15, value:60 }, { name:'tin', weight:68, value:45 }, { name:'banana', weight:27, value:60 }, { name:'apple', weight:39, value:40 }], 100) should return 405.

knapsack([{ name:'map', weight:9, value:150 }, { name:'compass', weight:13, value:35 }, { name:'water', weight:153, value:200 }, { name:'sandwich', weight:50, value:160 }, { name:'glucose', weight:15, value:60 }, { name:'tin', weight:68, value:45 }, { name:'banana', weight:27, value:60 }, { name:'apple', weight:39, value:40 }], 200) should return 510.

knapsack([{ name:'cheese', weight:23, value:30 }, { name:'beer', weight:52, value:10 }, { name:'suntan cream', weight:11, value:70 }, { name:'camera', weight:32, value:30 }, { name:'T-shirt', weight:24, value:15 }, { name:'trousers', weight:48, value:10 }, { name:'umbrella', weight:73, value:40 }], 100) should return 145.

knapsack([{ name:'cheese', weight:23, value:30 }, { name:'beer', weight:52, value:10 }, { name:'suntan cream', weight:11, value:70 }, { name:'camera', weight:32, value:30 }, { name:'T-shirt', weight:24, value:15 }, { name:'trousers', weight:48, value:10 }, { name:'umbrella', weight:73, value:40 }], 200) should return 185.

knapsack([{ name:'waterproof trousers', weight:42, value:70 }, { name:'waterproof overclothes', weight:43, value:75 }, { name:'note-case', weight:22, value:80 }, { name:'sunglasses', weight:7, value:20 }, { name:'towel', weight:18, value:12 }, { name:'socks', weight:4, value:50 }, { name:'book', weight:30, value:10 }], 100) should return 237.

knapsack([{ name:'waterproof trousers', weight:42, value:70 }, { name:'waterproof overclothes', weight:43, value:75 }, { name:'note-case', weight:22, value:80 }, { name:'sunglasses', weight:7, value:20 }, { name:'towel', weight:18, value:12 }, { name:'socks', weight:4, value:50 }, { name:'book', weight:30, value:10 }], 200) should return 317.' 
"""
test_case = list(map(lambda s: s.split('],')[0],
    test_case_1.replace('”', '"').replace('\n','').replace('knapsack([','').replace('}, {','};{').split('should return')
))[0]
print(test_case) 
wts = list(map(lambda s: int(s.split(',')[0]), test_case.split('weight:')[1:]))
vals = list(map(lambda s: int(s.split(' }')[0]), test_case.split('value:')[1:]))
w_lim = 100
print(wts, vals)

{ name:'map', weight:9, value:150 };{ name:'compass', weight:13, value:35 };{ name:'water', weight:153, value:200 };{ name:'sandwich', weight:50, value:160 };{ name:'glucose', weight:15, value:60 };{ name:'tin', weight:68, value:45 };{ name:'banana', weight:27, value:60 };{ name:'apple', weight:39, value:40 }
[9, 13, 153, 50, 15, 68, 27, 39] [150, 35, 200, 160, 60, 45, 60, 40]


In [103]:
def knapsack(vals, wts, W_lim):
    # record value in DP table
    assert len(wts) == len(vals) , "len must equal"
    n = len(wts)
    dp = np.zeros((n+1, W_lim+1))
    for i in range(1, n+1): # iterate number of items
        for j in range(1, W_lim+1):  # iterate weight  
            if wts[i-1] <= j: 
                # ensure that there is a value that object can be added from
                dp[i][j] = max(
                    dp[i-1][j],  # value without the item
                    dp[i-1][j-wts[i-1]] + vals[i-1]  # value with the new object
                )
            else:
                dp[i][j] =  dp[i-1][j]
            # print(dp[i][j])
    return dp[len(vals)][W_lim]


print(knapsack(vals, wts, w_lim))

405.0


## string brackets

In [74]:
def validate_string_brackets(s):
  vocab = {
    "{":"}",
    "(":")",
    "[":"]"
  }
  vocab_rev = {v:k for k,v in vocab.items()}
  queue = []
  for c in s:
    if c in vocab:
      queue.append(c)
    else:
      if c in vocab_rev:
        if len(queue) != 0 and queue[-1] == vocab_rev[c]:
          queue.pop(-1)
        else: return False
      elif c == " ":
        continue
      else: return False
  return True

test_case_2 = '( ) [ ] { } '
test_case_2b = '( [ ) ]'
test_case_2c = '{ [ ( ) ] }'
validate_string_brackets(test_case_2c)

True

## Video Editing 

In [75]:
import cv2

In [86]:
def add_red_box(frame, ori=(0, 0), size=(100, 100)):
  """ori: (x, y) {width, height}"""
  cv2.rectangle(frame, ori, (ori[0]+size[0], ori[1]+size[1]), (255,0,0), 5)
  return frame

def edit_video(vid_fp, t_frame = (30, 35), box_size=(100, 100)):
  cap = cv2.VideoCapture(vid_fp)
  ret, frame = cap.read()
  h, w, _ = frame.shape
  print(f"Frameshape:{frame.shape}")
  #
  fps, frame_count = cap.get(cv2.CAP_PROP_FPS), int(cap.get(cv2.CAP_PROP_FRAME_COUNT))      # 
  duration = frame_count/fps
  print(f"Duration:{duration}\nFPS:{fps}")
  # Write /clip vieo
  fourcc = cv2.VideoWriter_fourcc(*"XVID")
  out_fp = f"{vid_fp.split('.')[0]}_t{t_frame[0]}-{t_frame[1]}s_edited.avi"
  writer = cv2.VideoWriter(f"{out_fp}", fourcc, fps, (w, h))
  f = 0
  while ret:
    f += 1
    if t_frame[0]*fps <= f <= t_frame[1]*fps:
      frame = add_red_box(frame, ori=(w//2-box_size[0]//2, h//2-box_size[1]//2))
      writer.write(frame)
    ret, frame = cap.read()
  writer.release()
  return out_fp

file = "test_video.mp4"

edit_video(file, t_frame=(30,35))

Frameshape:(360, 640, 3)
Duration:101.88
FPS:25.0


'test_video_t30-35s.avi'